### 時間序列新增  7 日 搭乘量 (7DaysTimes)
### 時間序列次數改為 移動平均 (MATimes)

In [1]:
import time
import pandas as pd
import datetime
import os

In [2]:
# 檔案讀取位置
datasite = './dfTime_TLC-201902_202306.parquet'

# 是否輸出檔案 >0 輸出
df_save = 1
# 檔案輸出位置 
custom_output_path = f'./7DaysTimes'
DataName = '201902_202306'

#設定 {N+M+1}DaysTimes 天數 
N = 3 #往前天數
M = 3 #往後天數

#-------------------------------------------------------------------------------------------------#
# 記錄開始時間
start_time = time.time()

df = pd.read_parquet(datasite)

# 記錄結束時間
end_time = time.time()   
# 計算程式執行花費的時間
execution_time = end_time - start_time

print("程式執行花費的時間：", round(execution_time,2), "秒")
print("done時間:", datetime.datetime.now())
df

程式執行花費的時間： 2.32 秒
done時間: 2023-08-24 17:26:02.509781


,Name,year,month,day,hour,weekday,PULocationID,is_holiday,次數
0,lyft,2019,2,1,0,4,3,False,4
1,lyft,2019,2,1,0,4,4,False,22
2,lyft,2019,2,1,0,4,6,False,1
3,lyft,2019,2,1,0,4,7,False,50
4,lyft,2019,2,1,0,4,9,False,2
...,...,...,...,...,...,...,...,...,...
22870509,yellow,2023,6,30,23,4,255,False,7
22870510,yellow,2023,6,30,23,4,256,False,7
22870511,yellow,2023,6,30,23,4,261,False,19
22870512,yellow,2023,6,30,23,4,262,False,18


### 新增 前N後M日搭乘量 (NDaysTimes)
    預設前3後3 總共7天

In [3]:
# 記錄開始時間
start_time = time.time()

#創立一個 df_temp 統計N天內總次數
df['TempTime'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].astype(str)+'-' +df['day'].astype(str),
                                format='%Y-%m-%d')
df_temp = df[['Name','PULocationID','TempTime','次數']]
df_temp = df_temp.groupby([ 'Name','TempTime','PULocationID'])['次數'].sum().reset_index()

TempTime_unique_count = df['TempTime'].nunique()
print(f'總天數:{TempTime_unique_count}')
temp = 0
temp1 = '0'

for index, row in df_temp.iterrows():
    temp_start_time = row['TempTime'] - pd.DateOffset(days=N)
    temp_end_time = row['TempTime'] + pd.DateOffset(days=M)
    temp_count = df_temp.loc[(df_temp['TempTime'] >= temp_start_time) & (df_temp['TempTime'] <= temp_end_time) & (row['Name'] == df_temp['Name']) & (row['PULocationID'] == df_temp['PULocationID']) ]['次數'].sum()
    temp_day =  row['TempTime'] - df_temp['TempTime'].min()
    if row['TempTime'].day == 1:  # 每個月的1日印出來
        if (temp != row['TempTime']) | (temp1  != row['Name']) :
            temp = row['TempTime']
            temp1 = row['Name']
            print(row['Name'],row['TempTime'])
    if TempTime_unique_count - temp_day.days <= M:          
        temp_count =  df_temp.loc[(df_temp['TempTime'] >= df_temp['TempTime'].max() -pd.DateOffset(days=N)-pd.DateOffset(days=M)) & (df_temp['TempTime'] <= df_temp['TempTime'].max() ) & (row['Name'] == df_temp['Name']) & (row['PULocationID'] == df_temp['PULocationID']) ]['次數'].sum()
        df_temp.at[index,f'{N+1+M}DaysTimes'] = temp_count
    elif temp_day.days >= N:  
        df_temp.at[index,f'{N+1+M}DaysTimes'] = temp_count
    else : #處理前N-1天
        temp_count =  df_temp.loc[(df_temp['TempTime'] >= df_temp['TempTime'].min()) & (df_temp['TempTime'] <= ( df_temp['TempTime'].min() +pd.DateOffset(days=N)+pd.DateOffset(days=M)) ) & (row['Name'] == df_temp['Name']) & (row['PULocationID'] == df_temp['PULocationID']) ]['次數'].sum()
        #temp_count = temp_count *(N+1) / (temp_day.days+1)        
        df_temp.at[index,f'{N+1+M}DaysTimes'] = temp_count
        
del temp_start_time,temp_end_time,temp_count,temp_day,temp,temp1  

# 放回 df
df[f'{N+1+M}DaysTimes'] = df.apply(lambda row:df_temp.loc[(df_temp['Name'] == row['Name']) &
                                                        (df_temp['TempTime'] == row['TempTime']),
                                                        f'{N+1+M}DaysTimes'].values[0], axis=1)
# TempTime移除
df = df.drop(columns=['TempTime'])
#df['TempTime'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].astype(str)+'-' +df['day'].astype(str) + ' ' + df['hour'].astype(str) ,format='%Y-%m-%d %H')

# 輸出檔案
if df_save > 0 :
    os.makedirs(os.path.dirname(f'{custom_output_path}/dfTime_TLC-{DataName}.parquet'), exist_ok=True)
    df.to_parquet(f'{custom_output_path}/dfTime_TLC-{DataName}.parquet')
    print('save!')
    
# 記錄結束時間
end_time = time.time()   
# 計算程式執行花費的時間
execution_time = end_time - start_time
print("\n程式執行花費的時間：", round(execution_time,2), "秒")
print("done時間:", datetime.datetime.now())

總天數:1611
lyft 2019-02-01 00:00:00
lyft 2019-03-01 00:00:00
lyft 2019-04-01 00:00:00
lyft 2019-05-01 00:00:00
lyft 2019-06-01 00:00:00
lyft 2019-07-01 00:00:00
lyft 2019-08-01 00:00:00
lyft 2019-09-01 00:00:00
lyft 2019-10-01 00:00:00
lyft 2019-11-01 00:00:00
lyft 2019-12-01 00:00:00
lyft 2020-01-01 00:00:00
lyft 2020-02-01 00:00:00
lyft 2020-03-01 00:00:00
lyft 2020-04-01 00:00:00
lyft 2020-05-01 00:00:00
lyft 2020-06-01 00:00:00
lyft 2020-07-01 00:00:00
lyft 2020-08-01 00:00:00
lyft 2020-09-01 00:00:00
lyft 2020-10-01 00:00:00
lyft 2020-11-01 00:00:00
lyft 2020-12-01 00:00:00
lyft 2021-01-01 00:00:00
lyft 2021-02-01 00:00:00
lyft 2021-03-01 00:00:00
lyft 2021-04-01 00:00:00
lyft 2021-05-01 00:00:00
lyft 2021-06-01 00:00:00
lyft 2021-07-01 00:00:00
lyft 2021-08-01 00:00:00
lyft 2021-09-01 00:00:00
lyft 2021-10-01 00:00:00
lyft 2021-11-01 00:00:00
lyft 2021-12-01 00:00:00
lyft 2022-01-01 00:00:00
lyft 2022-02-01 00:00:00
lyft 2022-03-01 00:00:00
lyft 2022-04-01 00:00:00
lyft 2022-05-01 

KeyboardInterrupt: 

In [33]:
df[df['PULocationID']==2].head(10)

,Name,year,month,day,hour,weekday,PULocationID,is_holiday,次數,7DaysTimes
23570,lyft,2019,2,5,4,1,1,False,1,2.0
25446,lyft,2019,2,5,12,1,1,False,1,2.0
54535,lyft,2019,2,10,15,6,1,False,1,2.0
63874,lyft,2019,2,12,7,1,1,False,1,4.0
83159,lyft,2019,2,15,16,4,1,False,1,3.0
84140,lyft,2019,2,15,20,4,1,False,1,3.0
114380,lyft,2019,2,21,3,3,1,False,1,1.0
137495,lyft,2019,2,25,4,0,1,False,1,1.0
161230,uber,2019,2,1,8,4,1,False,2,4.0
161483,uber,2019,2,1,9,4,1,False,1,4.0


In [34]:
#檢查用
df_temp[df_temp['PULocationID']==2].head(10)

,Name,TempTime,PULocationID,次數,7DaysTimes
2816,lyft,2019-02-12,2,1,2.0
3328,lyft,2019-02-14,2,1,2.0
12547,lyft,2019-03-22,2,1,4.0
12805,lyft,2019-03-23,2,1,4.0
13061,lyft,2019-03-24,2,1,4.0
13317,lyft,2019-03-25,2,1,4.0
16389,lyft,2019-04-06,2,1,1.0
17926,lyft,2019-04-12,2,1,2.0
18439,lyft,2019-04-14,2,1,3.0
18952,lyft,2019-04-16,2,1,3.0


In [35]:
df['TempTime'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].astype(str)+'-' +df['day'].astype(str),format='%Y-%m-%d')
df = df.drop(columns=['7DaysTimes'])
merged_df = df.merge(df_temp[['Name', 'TempTime','PULocationID', f'{N+1+M}DaysTimes']], on=['Name', 'TempTime','PULocationID'], how='left')
df[f'{N+1+M}DaysTimes'] = merged_df[f'{N+1+M}DaysTimes']
del merged_df
df = df.drop(columns=['TempTime'])

# 輸出檔案
if df_save > 0 :
    os.makedirs(os.path.dirname(f'{custom_output_path}/dfTime_TLC-{DataName}.parquet'), exist_ok=True)
    df.to_parquet(f'{custom_output_path}/dfTime_TLC-{DataName}.parquet')
    print('save!')

save!
